# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

vacay_df=pd.read_csv('output_data/clean_city_data.csv').reset_index()
vacay_df.drop(["level_0","Unnamed: 0", "index"], axis=1, inplace=True)
vacay_df.head()

,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Punta Arenas,-53.1500,-70.9167,59.00,41,40,21.85,CL,02/06/21 13:10
1,Richards Bay,-28.7830,32.0377,75.31,80,25,8.16,ZA,02/06/21 13:18
2,Qaanaaq,77.4840,-69.3632,12.22,88,100,8.16,GL,02/06/21 13:16
3,Mar del Plata,-38.0023,-57.5575,69.80,64,75,14.97,AR,02/06/21 13:16
4,Rikitea,-23.1203,-134.9692,77.65,74,100,12.37,PF,02/06/21 13:15


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = vacay_df[["Latitude", "Longitude"]].astype(float)
humidity=vacay_df["Humidity (%)"].astype(float)
locations

,Latitude,Longitude
0,-53.1500,-70.9167
1,-28.7830,32.0377
2,77.4840,-69.3632
3,-38.0023,-57.5575
4,-23.1203,-134.9692
...,...,...
561,-32.2500,115.7667
562,52.0508,44.3451
563,-34.6000,138.7333
564,50.2834,-107.8014


In [4]:
#Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN",center=(2.0, -40.0), zoom_level=3)

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,max_intensity=100,point_radius=3)

#add layer
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather_df=vacay_df[(vacay_df["Max Temperature (F)"]<=84) & 
                        (vacay_df["Max Temperature (F)"]>=72) & 
                        (vacay_df["Wind Speed (mph)"]<10) & 
                        (vacay_df["Cloudiness (%)"]<=10)].reset_index()
ideal_weather_df.iloc[0:10].to_csv("output_data/ideal_weather_data.csv")
ideal_weather_df.iloc[0:10]

,index,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,40,Tocopilla,-22.0920,-70.1979,73.81,53,7,6.33,CL,02/06/21 13:16
1,51,Beloha,-25.1667,45.0500,77.86,81,7,5.68,MG,02/06/21 13:16
2,54,Cabo San Lucas,22.8909,-109.9124,78.01,56,1,6.91,MX,02/06/21 13:14
3,78,Noumea,-22.2763,166.4572,77.00,100,0,2.30,NC,02/06/21 13:14
4,121,Castro,-24.7911,-50.0119,76.44,48,0,6.96,BR,02/06/21 13:15
5,127,Isangel,-19.5500,169.2667,77.49,80,0,3.76,VU,02/06/21 13:16
6,141,Singarāyakonda,15.2500,80.0333,72.50,70,5,6.85,IN,02/06/21 13:19
7,143,Coahuayana Viejo,18.7333,-103.6833,75.20,64,1,5.03,MX,02/06/21 13:19
8,190,Port Blair,11.6667,92.7500,78.85,76,4,7.92,IN,02/06/21 13:17
9,227,Socoltenango,16.2167,-92.2500,76.28,38,3,7.65,MX,02/06/21 13:19


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#hotel_name=[]
#hotel_df=pd.DataFrame({"Hotel Name":hotel_name})
ideal_weather_df["Hotel Name"]=""
hotel_df=ideal_weather_df.iloc[0:10]
hotel_df

,index,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
0,40,Tocopilla,-22.0920,-70.1979,73.81,53,7,6.33,CL,02/06/21 13:16,
1,51,Beloha,-25.1667,45.0500,77.86,81,7,5.68,MG,02/06/21 13:16,
2,54,Cabo San Lucas,22.8909,-109.9124,78.01,56,1,6.91,MX,02/06/21 13:14,
3,78,Noumea,-22.2763,166.4572,77.00,100,0,2.30,NC,02/06/21 13:14,
4,121,Castro,-24.7911,-50.0119,76.44,48,0,6.96,BR,02/06/21 13:15,
5,127,Isangel,-19.5500,169.2667,77.49,80,0,3.76,VU,02/06/21 13:16,
6,141,Singarāyakonda,15.2500,80.0333,72.50,70,5,6.85,IN,02/06/21 13:19,
7,143,Coahuayana Viejo,18.7333,-103.6833,75.20,64,1,5.03,MX,02/06/21 13:19,
8,190,Port Blair,11.6667,92.7500,78.85,76,4,7.92,IN,02/06/21 13:17,
9,227,Socoltenango,16.2167,-92.2500,76.28,38,3,7.65,MX,02/06/21 13:19,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "22.2855,114.1577",  # philadelphia coords
    "radius":5000,
    "type": "lodging",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    lati=hotel_df.loc[index,"Latitude"]
    longi=hotel_df.loc[index,"Longitude"]
    lt_lng=f"{lati},{longi}"
    params["location"]=lt_lng
    hotel_data=requests.get(base_url,params=params).json()
    
    try:
        hotel_df.loc[index,"Hotel Name"]=hotel_data['results'][0]['name']
    except IndexError:
        hotel_df.loc[index, "Hotel Name"]="NaN"

hotel_df_cpy=hotel_df.loc[hotel_df["Hotel Name"]!="NaN"]
hotel_df.iloc[0:10].to_csv("output_data/hotel_data.csv")
hotel_df_cpy=hotel_df_cpy.iloc[0:10]
hotel_df_cpy

C:\Users\Silho\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,index,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
0,40,Tocopilla,-22.0920,-70.1979,73.81,53,7,6.33,CL,02/06/21 13:16,Hotel Galvarino
2,54,Cabo San Lucas,22.8909,-109.9124,78.01,56,1,6.91,MX,02/06/21 13:14,Hotel Tesoro Los Cabos
3,78,Noumea,-22.2763,166.4572,77.00,100,0,2.30,NC,02/06/21 13:14,Hôtel Le Lagon
4,121,Castro,-24.7911,-50.0119,76.44,48,0,6.96,BR,02/06/21 13:15,CHACARA BAILLY
5,127,Isangel,-19.5500,169.2667,77.49,80,0,3.76,VU,02/06/21 13:16,Volcano Sea View Tree House
6,141,Singarāyakonda,15.2500,80.0333,72.50,70,5,6.85,IN,02/06/21 13:19,srinu tiffin shop
7,143,Coahuayana Viejo,18.7333,-103.6833,75.20,64,1,5.03,MX,02/06/21 13:19,Hotel Los Arcos
8,190,Port Blair,11.6667,92.7500,78.85,76,4,7.92,IN,02/06/21 13:17,Welcomhotel Bay Island Port Blair
9,227,Socoltenango,16.2167,-92.2500,76.28,38,3,7.65,MX,02/06/21 13:19,Canto del Agua Ecolodge


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_cpy.iterrows()]
hotel_locations = hotel_df_cpy[["Latitude", "Longitude"]]

# Add marker layer ontop of heat map
markers=gmaps.marker_layer(hotel_locations,info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))